In [ ]:
!pip install astroquery
import os
import io
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload
from astroquery.sdss import SDSS
from astropy import coordinates as coords
from astropy.io import fits
from astroquery.sdss import SDSS
from astropy import coordinates as coords
import pandas as pd
!pip install --upgrade astroquery

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
csv_file_path = "/content/drive/MyDrive/AstroResearch/new_data_table/Optical_spectral_types_DR17_v3_ctheissen.csv"
df = pd.read_csv(csv_file_path)
df

In [ ]:
def degrees_to_hours_minutes_seconds(ra_degrees):
    ra_hours = ra_degrees / 15
    hours = int(ra_hours)
    minutes = (ra_hours - hours) * 60
    seconds = (minutes - int(minutes)) * 60
    return hours, int(minutes), seconds

def degrees_to_degrees_arcminutes_arcseconds(dec_degrees):
    degrees = int(dec_degrees)
    arcminutes = (dec_degrees - degrees) * 60
    arcseconds = (arcminutes - int(arcminutes)) * 60
    return degrees, int(arcminutes), arcseconds

# Example usage
ra_degrees = 196.713400  # replace with your RA in degrees
dec_degrees = 0.298453  # replace with your Dec in degrees

ra_hms = degrees_to_hours_minutes_seconds(ra_degrees)
dec_dms = degrees_to_degrees_arcminutes_arcseconds(dec_degrees)

print(ra_hms)
print(dec_dms)

(13, 6, 51.21600000000171)
(0, 17, 54.43080000000002)


In [ ]:
count = df['bestObjID'].value_counts()
count

In [ ]:
specific_id = 1237661385995124738
rows_with_id = df[df['bestObjID'] == specific_id]
rows_with_id.head()

In [ ]:
mjd_count = rows_with_id['mjd'].value_counts()
mjd_count

In [ ]:
# drop the rows with the same ID
df_unique = df.drop_duplicates(subset='bestObjID', keep='first').reset_index(drop=True)
df_unique
# choose the best signal to noise
# keep the repeated ones and check the results

## number of rows removed:
# 140908 - 117966 = 22942

,mjd,plate,fiberID,bestObjID,ra,dec,class,subclass,instrument,rChi2,...,psfMagErr_u,psfMagErr_g,psfMagErr_r,psfMagErr_i,psfMagErr_z,extinction_u,extinction_g,extinction_r,extinction_i,extinction_z
0,56727,7456,532,1237648674511454293,196.713400,0.298453,STAR,A4V (97603),BOSS,1.881113,...,0.012358,0.011815,0.016474,0.024657,0.029614,0.071528,0.055734,0.038557,0.028652,0.021312
1,56727,7456,501,1237648674511454356,196.624950,0.282299,STAR,K3V (32147),BOSS,1.101248,...,0.070880,0.014013,0.016380,0.014305,0.023413,0.080341,0.062601,0.043307,0.032182,0.023937
2,56727,7456,531,1237648674511585290,196.900460,0.240210,STAR,G0 (63791),BOSS,1.112199,...,0.023038,0.014827,0.012288,0.014434,0.023888,0.083197,0.064826,0.044847,0.033326,0.024788
3,56727,7456,563,1237648674511585391,196.968910,0.349134,STAR,K3V (32147),BOSS,1.346109,...,0.039587,0.015189,0.012435,0.014620,0.024842,0.066900,0.052128,0.036062,0.026798,0.019933
4,56727,7456,571,1237648674511650951,197.030410,0.239345,STAR,K5Ve (118100),BOSS,3.276671,...,0.027482,0.015707,0.012603,0.013149,0.022902,0.073882,0.057568,0.039826,0.029595,0.022013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117961,57364,7715,622,1237680311769301325,8.753181,33.065267,STAR,G8V (101501),BOSS,1.072950,...,0.052898,0.016856,0.011773,0.013551,0.023144,0.338356,0.263645,0.182388,0.135534,0.100812
117962,56561,6293,346,1237680331096981717,343.825470,27.206893,STAR,F3/F5V (30743),BOSS,2.364043,...,0.018976,0.016089,0.011750,0.013114,0.026265,0.247947,0.193198,0.133654,0.099319,0.073875
117963,56566,6292,496,1237680331634376734,344.995420,27.781598,STAR,F3/F5V (30743),BOSS,1.104141,...,0.022101,0.014612,0.017602,0.028957,0.018391,0.254428,0.198248,0.137147,0.101915,0.075806
117964,57362,7718,44,1237680331644993617,12.683085,29.070565,STAR,F0IV (81937),BOSS,1.077174,...,0.018712,0.016770,0.013694,0.024185,0.021729,0.244040,0.190154,0.131548,0.097754,0.072711


In [ ]:
file_path = 'no_spectral_IDs.txt'  # Replace 'path/to/your/file.txt' with the actual file path

# Initialize an empty list to store the lines from the file
lines_list = []

# Open the file in read mode and read each line
with open(file_path, 'r') as file:
    for line in file:
        # Remove any leading/trailing whitespaces (e.g., newlines, spaces)
        clean_line = line.strip()
        lines_list.append(clean_line)

# Now the lines from the file are stored in the 'lines_list' list
lines_list
len(lines_list)

18030

In [ ]:
# Sample list of strings representing numbers
string_numbers = lines_list

no_spec_IDs = list(map(int, string_numbers))# Convert the strings to integers using map function
no_spec_IDs

In [ ]:
unique_values_count = len(set(no_spec_IDs))
unique_values_count

18009

In [ ]:
ids_to_filter = no_spec_IDs
filtered_df = df_unique[df_unique['bestObjID'].isin(ids_to_filter)].reset_index() # Filter the DataFrame to keep only the rows with matching IDs
filtered_df

,index,mjd,plate,fiberID,bestObjID,ra,dec,class,subclass,instrument,...,psfMagErr_u,psfMagErr_g,psfMagErr_r,psfMagErr_i,psfMagErr_z,extinction_u,extinction_g,extinction_r,extinction_i,extinction_z
0,1280,57742,9363,47,1237651190811722019,113.950650,34.372126,STAR,F3/F5V (30743),BOSS,...,0.017464,0.012539,0.009170,0.011710,0.016594,0.308535,0.240408,0.166313,0.123589,0.091927
1,1405,58930,12545,494,1237648720138928174,138.685690,-0.958741,STAR,F3/F5V (30743),BOSS,...,0.022141,0.014021,0.015055,0.012437,0.026831,0.156954,0.122298,0.084605,0.062871,0.046764
2,1406,58930,12545,495,1237648720139059421,139.073220,-0.856066,STAR,A2II (39866),BOSS,...,0.021830,0.025277,0.014444,0.012289,0.022772,0.166806,0.129974,0.089916,0.066817,0.049700
3,1411,57817,9368,449,1237650340407148759,115.612040,37.052351,STAR,M4.5:III (123657),BOSS,...,0.862787,0.479133,0.156744,0.092413,0.080648,0.291733,0.227317,0.157257,0.116859,0.086921
4,1469,58488,11068,703,1237649921115881523,21.352642,15.355068,STAR,F3/F5V (30743),BOSS,...,0.029868,0.018641,0.025499,0.010451,0.019169,0.180175,0.140391,0.097122,0.072172,0.053683
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18004,117925,58097,7716,30,1237680478734712949,12.513824,32.577455,STAR,F3/F5V (30743),BOSS,...,0.021616,0.013938,0.012181,0.012690,0.011896,0.320113,0.249429,0.172554,0.128226,0.095377
18005,117928,58097,7716,398,1237680479270797727,10.312414,33.171030,STAR,M5III (221615),BOSS,...,0.585710,0.030942,0.018510,0.015598,0.017280,0.324558,0.252893,0.174950,0.130007,0.096701
18006,117930,58097,7716,854,1237680479808062003,11.498197,33.439076,STAR,M3III (44478),BOSS,...,0.362134,0.050351,0.025477,0.016289,0.028152,0.324193,0.252609,0.174754,0.129861,0.096593
18007,117931,58097,7716,608,1237680511482986978,10.149730,33.697313,STAR,G1V (95128),BOSS,...,0.151211,0.025408,0.022134,0.027865,0.045407,0.356163,0.277520,0.191987,0.142667,0.106118


In [ ]:
import pandas as pd
df = filtered_df

# Create a txt file called "download_rsync.txt" and write the content
with open("download_rsync.txt", "w") as file:
    for _, row in df.iterrows():
        plate = row['plate']
        mjd = row['mjd']
        fiber_id = row['fiberID']
        line = f"/eboss/spectro/redux/v5_13_0/spectra/lite/{plate}/spec-{plate}-{mjd}-{fiber_id}.fits\n"
        file.write(line)


In [ ]:
import pandas as pd

df = filtered_df

# Create a txt file called "download_rsync.txt" and write the content
with open("download_rsync.txt", "w") as file:
    for _, row in df.iterrows():
        plate = row['plate']
        mjd = row['mjd']
        fiber_id = str(row['fiberID']).zfill(4)
        line = f"/eboss/spectro/redux/v5_13_0/spectra/lite/{plate}/spec-{plate}-{mjd}-{fiber_id}.fits\n"
        file.write(line)


## query without parallel-processing

In [ ]:
start_index =
end_index =

bestObjID_list = df_unique['bestObjID'].iloc[start_index-1:end_index].tolist()
bestObjID_list[-1]

1237680499672089472

In [ ]:
bestObjID_list=[1237649954404630667]

In [ ]:
start_index =
end_index =

bestObjID_list = df_unique['bestObjID'].iloc[start_index-1:end_index].tolist()
bestObjID_list[-1]


# Define a list of object IDs
object_ids = bestObjID_list  # Replace with my own list

# Create a folder to save the spectra files in Google Drive
output_dir = "/content/drive/MyDrive/AstroResearch/new_data_table/spectra_files_Gavin"
os.makedirs(output_dir, exist_ok=True)

# Create a text file for logging in Google Drive
with open("/content/drive/MyDrive/AstroResearch/new_data_table/no_spectral_recording_new.txt", "a") as log_file: #(a: let us append; w: let us overwrite)

    # Query and download the spectra files for each object ID
    for object_id in object_ids:

        # Query the object's coordinates using the object ID
        query = f"SELECT ra, dec FROM PhotoObj WHERE objid = {object_id}"
        result = SDSS.query_sql(query)
        print(result)
        #if there is a coordinate, then query the spectrum
        if result is not None and len(result) > 0:
            ra, dec = result[0]['ra'], result[0]['dec']

            # Query the spectrum using the coordinates
            co = coords.SkyCoord(ra, dec, unit=('deg', 'deg'), frame='icrs')
            xid = SDSS.query_region(co, spectro=True)

            #if there is a spectrum, then download the spectrum
            if xid is not None and len(xid) > 0:
                sp = SDSS.get_spectra(matches=xid)   # Download the spectrum as a FITS file
                if sp is not None and len(sp) > 0:
                    file_name = f"{output_dir}/spectrum_{object_id}.fits" # save in google drive
                    sp[0].writeto(file_name, overwrite=True)
                    print(f"Downloaded and saved spectrum for object ID {object_id} as {file_name}")
                else:
                    print(f"No spectrum available for object ID {object_id}\n")
            else:
                print(f"No spectral data found for object ID {object_id}\n")
        else:
            print(f"No object found with object ID {object_id}\n")


In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u

# Define a list of object IDs
object_ids = [1237649954404630667]  # Replace with your own list

# Create a folder to save the spectra files in Google Drive
output_dir = "/content/drive/MyDrive/AstroResearch/new_data_table/spectra_files_Gavin"
os.makedirs(output_dir, exist_ok=True)

# Create a text file for logging in Google Drive
with open("/content/drive/MyDrive/AstroResearch/new_data_table/no_spectral_recording_new.txt", "a") as log_file:

    # Query and download the spectra files for each object ID
    for object_id in object_ids:

        # Query the object's coordinates using the object ID
        query = f"SELECT ra, dec FROM PhotoObj WHERE objid = {object_id}"
        result = SDSS.query_sql(query)

        # If there is a coordinate, then query the spectrum
        if result is not None and len(result) > 0:
            ra, dec = result[0]['ra'], result[0]['dec']

            # Query the spectrum using the coordinates and the specified radius
            co = SkyCoord(ra, dec, unit=('deg', 'deg'), frame='icrs')
            xid = SDSS.query_region(co, spectro=True, radius=6*u.arcsec) # Search within a radius of 6 arcseconds

            # If there is a spectrum, then download the spectrum
            if xid is not None and len(xid) > 0:
                sp = SDSS.get_spectra(matches=xid)   # Download the spectrum as a FITS file
                if sp is not None and len(sp) > 0:
                    file_name = f"{output_dir}/spectrum_{object_id}.fits" # save in Google Drive
                    sp[0].writeto(file_name, overwrite=True)
                    print(f"Downloaded and saved spectrum for object ID {object_id} as {file_name}")
                else:
                    log_file.write(f"No spectrum available for object ID {object_id}\n")
            else:
                log_file.write(f"No spectral data found for object ID {object_id}\n")
        else:
            log_file.write(f"No object found with object ID {object_id}\n")

In [ ]:
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as u

# Query and download the spectra files for each object ID
for object_id in [1237649954406924514]: # successful one: 1237648674511585290 # not succ: 1237649921119092964
    # Query the object's coordinates using the object ID
    query = f"SELECT ra, dec FROM PhotoObj WHERE objid = {object_id}"
    result = SDSS.query_sql(query)
    print(result)

    if result is not None and len(result) > 0:
        ra, dec = result[0]['ra'], result[0]['dec']
        co = SkyCoord(ra, dec, unit=('deg', 'deg'), frame='icrs')
        xid = SDSS.query_region(co, spectro=True, radius= 100 * u.arcsec)
        print(co)
        print(xid)

        if xid is not None and len(xid) > 0:
            # Create SkyCoord objects for the matches
            match_coords = SkyCoord(xid['ra'], xid['dec'], unit=('deg', 'deg'), frame='icrs')

            # Find the index of the closest match using match_coordinates_sky
            idx, _, _ = match_coordinates_sky(co, match_coords)

            # Download the spectrum of the closest match
            closest_spectrum = SDSS.get_spectra(matches=xid[idx:idx+1])
            if closest_spectrum is not None and len(closest_spectrum) > 0:
                file_name = f"{output_dir}/spectrum_{object_id}.fits"
                closest_spectrum[0].writeto(file_name, overwrite=True)
                print(f"Downloaded and saved spectrum for object ID {object_id} as {file_name}")
            else:
                print(f"No spectrum available for object ID {object_id}\n")
        else:
            print(f"No spectral data found for object ID {object_id}\n")
    else:
        print(f"No object found with object ID {object_id}\n")


       ra              dec       
---------------- ----------------
27.6753509765275 14.3003183990828
<SkyCoord (ICRS): (ra, dec) in deg
    (27.67535098, 14.3003184)>
None
No spectral data found for object ID 1237649954406924514



## Query with parallel-processing

In [ ]:
from google.colab import drive
from concurrent.futures import ThreadPoolExecutor
import os
from astropy import coordinates as coords
from astroquery.sdss import SDSS

In [ ]:
start_index =
end_index =

bestObjID_list = df_unique['bestObjID'].iloc[start_index-1:end_index].tolist()
bestObjID_list[-1]

1237661386539204683

In [ ]:
# Define a list of object IDs
object_ids = bestObjID_list  # Replace with my own list
# Create a folder to save the spectra files in Google Drive
output_dir = "/content/drive/MyDrive/AstroResearch/new_data_table/spectra_files40001-50000"
os.makedirs(output_dir, exist_ok=True)

# Define a function to be run in parallel for each object ID
def process_object_id(object_id):
    # Query the object's coordinates using the object ID
    query = f"SELECT ra, dec FROM PhotoObj WHERE objid = {object_id}"
    result = SDSS.query_sql(query)

    if result is not None and len(result) > 0:
        ra, dec = result[0]['ra'], result[0]['dec']

        # Query the spectrum using the coordinates
        co = coords.SkyCoord(ra, dec, unit=('deg', 'deg'), frame='icrs')
        xid = SDSS.query_region(co, spectro=True)

        if xid is not None and len(xid) > 0:
            # Download the spectrum as a FITS file
            sp = SDSS.get_spectra(matches=xid)
            if sp is not None and len(sp) > 0:
                # Save the downloaded spectrum to the Google Drive directory
                file_name = f"{output_dir}/spectrum_{object_id}.fits"
                sp[0].writeto(file_name, overwrite=True)
                print(f"Downloaded and saved spectrum for object ID {object_id} as {file_name}")  # Print out the success message
                return None
            else:
                return f"No spectrum available for object ID {object_id}"
        else:
            return f"No spectral data found for object ID {object_id}"
    else:
        return f"No object found with object ID {object_id}"

# Use ThreadPoolExecutor to run the function in parallel for each object ID
with ThreadPoolExecutor(max_workers=20) as executor:
    results = list(executor.map(process_object_id, object_ids))

# Create a text file for logging in Google Drive
# with open("/content/drive/MyDrive/AstroResearch/new_data_table/no_spectral_recording.txt", "a") as log_file: #(a: let us append; w: let us overwrite)
#     for result in results:
#         if result is not None:  # Only write to log if there was an error
#             log_file.write(result + '\n')
